In [37]:
%load_ext nb_black


The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [38]:
from elasticsearch import Elasticsearch

elasticsearch_url = "https://159.203.183.251:9200"
username = "pollett"
password = "9r0&rJP@19GY"
fingerprint = "CE:AA:F7:FF:04:C7:31:14:78:9C:62:D4:CE:98:F9:EF:56:DA:70:45:37:14:E3:F8:66:0A:25:ED:05:04:83:ec"



elastic_handle = Elasticsearch(
            hosts=elasticsearch_url,
            basic_auth=(username, password),
            ssl_assert_fingerprint=fingerprint,
        )



<IPython.core.display.Javascript object>

In [39]:
# Use the cat.indices API to list all index names
index_names = elastic_handle.indices.get(index="userpollinteractions", human=True)
index_names.body

# Extract the list of index names
#index_list = [index['index'] for index in index_names]
#index_list

{'userpollinteractions': {'aliases': {},
  'mappings': {'properties': {'userId': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'userPollActions': {'properties': {'actionType': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
      'createdAt': {'type': 'date'},
      'pollId': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}}}},
  'settings': {'index': {'creation_date_string': '2023-09-28T11:35:23.268Z',
    'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}},
    'number_of_shards': '1',
    'provided_name': 'userpollinteractions',
    'creation_date': '1695900923268',
    'number_of_replicas': '1',
    'uuid': 'kwSuQhk9QYuH5kycp7JtYA',
    'version': {'created_string': '8090199', 'created': '8090199'}}}}}

<IPython.core.display.Javascript object>

In [40]:
index_names = elastic_handle.cat.indices(h="index", format="json")

# Extract the list of index names
index_list = [index['index'] for index in index_names]

# Print the list of index names

index_list

['.internal.alerts-observability.metrics.alerts-default-000001',
 '.internal.alerts-observability.logs.alerts-default-000001',
 'userpollinteractions',
 '.internal.alerts-observability.uptime.alerts-default-000001',
 '.fleet-file-data-agent-000001',
 '.fleet-files-agent-000001',
 '.internal.alerts-security.alerts-default-000001',
 'polls',
 '.internal.alerts-observability.slo.alerts-default-000001',
 '.internal.alerts-observability.apm.alerts-default-000001']

<IPython.core.display.Javascript object>

In [41]:
def get_interactions(index_name, user_id, batch_size=100, es=elastic_handle):
        # setattr(self, index_name, [])
        # index_list = getattr(self, index_name)
        from_index = 0
        all_instances = []

        query = {
            "match_phrase": {"userId": user_id},
        }

        results = es.search( 
            index=index_name,
            query=query,
            size=batch_size,
            from_=from_index,
            timeout="1s",
        )
        # instances = results["hits"]["hits"][0]
        interactions = results["hits"].get("hits")
        interactions = interactions[0].get("_source").get("userPollActions")
        return interactions

def get_index(index_name, batch_size=100, es=elastic_handle):

        from_index = 0
        all_instances = []

        query ={"match_all": {}}
        
        while True:
            results = es.search(
                index=index_name,
                query=query,
                size=batch_size,
                from_=from_index,
                timeout="1s",
            )
            instances = results.get("hits").get("hits")

            all_instances.extend(instances)
            from_index += batch_size
            if len(instances) < 100:
                break

        all_instances = [instance["_source"] for instance in all_instances]
        return all_instances


<IPython.core.display.Javascript object>

In [42]:
polls = get_index(index_name="polls")
#for poll in polls:
#    print(poll)
len(polls)

2118

<IPython.core.display.Javascript object>

In [51]:
#[poll['pollLimitations'] for poll in polls]
[poll.get('pollLimitations') for poll in polls]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

<IPython.core.display.Javascript object>

In [19]:
interactions = get_interactions(index_name="userpollinteractions",
                 user_id="67eb27ca-ba0b-4d29-8627-9ec78327b512")
for hit in interactions:
    print(hit)
len(interactions)




{'createdAt': '2023-09-28T11:46:24.4138749Z', 'actionType': 'Like', 'pollId': 'b61f97c5-9d06-4e8d-a8f9-321f1fb0443d'}
{'createdAt': '2023-09-28T11:46:39.3766706Z', 'actionType': 'Like', 'pollId': 'c22c5989-2a32-4715-9c33-6ef39caf4f70'}
{'createdAt': '2023-09-28T11:47:09.3309395Z', 'actionType': 'Vote', 'pollId': '00763b59-dbdf-47fa-80a5-ec844bf28279'}
{'createdAt': '2023-09-28T11:47:24.3556975Z', 'actionType': 'Vote', 'pollId': '18d58c47-e3a2-408b-ad27-d612116459aa'}
{'createdAt': '2023-09-28T11:49:09.4662834Z', 'actionType': 'Vote', 'pollId': 'df5673cb-3e4e-454c-a82d-40813250f7c2'}
{'createdAt': '2023-09-28T11:49:24.5378566Z', 'actionType': 'Vote', 'pollId': '5e69ce0c-4309-4633-ba10-a78486d36b0b'}
{'createdAt': '2023-09-28T11:50:24.6840941Z', 'actionType': 'Vote', 'pollId': '3dc9cff3-184e-4764-860d-8c0a5dff5658'}
{'createdAt': '2023-09-28T11:50:39.7264643Z', 'actionType': 'Vote', 'pollId': '067545f9-d115-4384-8266-ff518b8ad1fa'}
{'createdAt': '2023-09-28T11:50:40.7021464Z', 'actionTyp

13

<IPython.core.display.Javascript object>

In [29]:

while True:
    # query = {"query": {"match_all": {}}, "size": batch_size, "from": from_index}
    results = self.client.search(
        index=index_name,
        query={"match_all": {}},
        size=batch_size,
        from_=from_index,
    )
    instances = results["hits"]["hits"]

    all_instances.extend(instances)
    from_index += batch_size
    if len(instances) < 100:
        break

setattr(self, index_name, [instance["_source"] for instance in all_instances])
return getattr(self, index_name)

NameError: name 'self' is not defined

<IPython.core.display.Javascript object>